# 数据的加载

### 从UCI官方加载数据集

In [ ]:
from ucimlrepo import fetch_ucirepo

# fetch dataset 
uci_dataset = fetch_ucirepo(id=45)

# data (as pandas dataframes) 
X = uci_dataset.data.features
y = uci_dataset.data.targets

# variable information 
print(uci_dataset.variables)

### 从UCI加载数据，转化格式，转换为数字形式，并保存为mat数据 
X为特征数据：num_instances,features
Y为lable：(num_instances,1) 是一个列向量

In [ ]:

import numpy as np
from sklearn.preprocessing import LabelEncoder
from ucimlrepo import fetch_ucirepo
from scipy.io import savemat

# 数据集
# 76 Nursery 
# 30 Contraceptive Method Choice
# 146 Satellite
# 33 Dermatology
# 23 Chess
# 19 Car
# 12 Balance Scale
# 39 Ecoli
# 42 Glass
# 78 Page Blocks
# 90 Soybean (Large)
# 69 Molecular Biology Splice-junction
# 59 Letter
uci_dataset = fetch_ucirepo(id=59)
# 1. 特征和标签分离
X = uci_dataset.data.features.values
X_columns = uci_dataset.data.features.columns.tolist()
y = uci_dataset.data.targets.values

In [ ]:
# 2. 特征编码
label_encoder = LabelEncoder()
X_encoded = np.copy(X)

# # 将特征数据每一列都进行编码
# for i in range(X.shape[1]):
#     X_encoded[:, i] = label_encoder.fit_transform(X_encoded[:, i])
y_encoded = label_encoder.fit_transform(y[:, 0])


#index=[163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184]
# 去除掉X_encoded、y_encoded中index里的索引对应的数据

def remove_data_by_index(X1, y1, index_list):
    """
    根据索引列表去除特征数据和标签数据中对应的数据。
    :param X1: 特征数据，类型为numpy.ndarray
    :param y1: 标签数据，类型为numpy.ndarray
    :param index_list: 索引列表，类型为list
    :return: 去除对应数据后的特征数据X2和标签数据y2
    """
    index_list = sorted(index_list, reverse=True)  # 先对索引列表排序，倒序方便后续删除
    for index in index_list:
        X1 = np.delete(X1, index, axis=0)  # 按行删除特征数据中对应索引的数据
        y1 = np.delete(y1, index, axis=0)  # 按行删除标签数据中对应索引的数据
    return X1, y1


#X_encoded,y_encoded=remove_data_by_index(X_encoded, y_encoded, index)
# 转成int格式
# X_encoded = X_encoded.astype(int)
#y_encoded = label_encoder.fit_transform(y_encoded)
y_encoded = y_encoded.astype(int)


In [ ]:
# 保存为 .mat 文件
data_dict = {'X': X_encoded, 'Y': y_encoded.reshape(-1, 1)}  # 以字典形式存储
savemat('Letter.mat', data_dict)
# 输出保存成功
print("数据保存成功！")

### 五折交叉验证

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.base import clone
import numpy as np


def k_fold_cross_validation_with_soft_labels(model, X, y, n_splits=5):
    """
    Perform 5-fold cross-validation and generate soft labels (probability predictions).

    Parameters:
    - model: A sklearn-compatible model with a `predict_proba` method.
    - X: Feature matrix (numpy array or pandas DataFrame).
    - y: Target vector (numpy array or pandas Series).

    Returns:
    - soft_labels: A numpy array containing the soft labels for each sample.
    - scores: A list of accuracy scores for each fold.
    """
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)  # 5-fold cross-validation
    soft_labels = np.zeros((len(y), len(np.unique(y))))  # Initialize array for soft labels
    scores = []
    for train_index, test_index in kf.split(X):
        # Split data into train and test
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        # Clone and fit the model on the training set
        model_clone = clone(model)
        model_clone.fit(X_train, y_train)
        # Generate soft labels (probability predictions)
        y_proba = model_clone.predict_proba(X_test)
        soft_labels[test_index] = y_proba

        # Evaluate the model
        y_pred = np.argmax(y_proba, axis=1)  # Convert probabilities to class predictions
        score = accuracy_score(y_test, y_pred)
        scores.append(score)

    return soft_labels, scores


# 示例用法
if __name__ == "__main__":
    from sklearn.datasets import load_iris
    from sklearn.ensemble import RandomForestClassifier

    # 加载数据集
    data = load_iris()
    X, y = data.data, data.target

    # 定义模型
    model = RandomForestClassifier(random_state=42)

    # 执行5折交叉验证
    soft_labels, scores = k_fold_cross_validation_with_soft_labels(model, X, y, n_splits=5)
    hard_labels = np.argmax(soft_labels, axis=1)
    # 输出结果
    print("Soft labels:", soft_labels)
    print("Hard labels:", hard_labels)
    print("Cross-validation scores:", scores)
    print("Mean accuracy:", np.mean(scores))


# 从imbalanced-learn官网加载UCI数据集

In [2]:
from imblearn.datasets import fetch_datasets
from sklearn.preprocessing import LabelEncoder
import numpy as np
from scipy.io import savemat
datasetname = 'sick_euthyroid'
imba_dataset = fetch_datasets()[datasetname]
print(imba_dataset.data.shape)
x = imba_dataset.data
y = imba_dataset.target
# 计算唯一元素的数量
print(np.unique(y, return_counts=True))

(3163, 42)
(array([-1,  1], dtype=int64), array([2870,  293], dtype=int64))


## 保存为mat文件

In [ ]:
# 2. 特征编码
label_encoder = LabelEncoder()
X_encoded = np.copy(x)

y_encoded = label_encoder.fit_transform(y)
# 保存为 .mat 文件
data_dict = {'X': X_encoded, 'Y': y_encoded.reshape(-1, 1)}  # 以字典形式存储
savemat(datasetname + '.mat', data_dict)
# 输出保存成功
print("数据保存成功！")